# Data Dictionary
**Variable** Meaning	Levels</br>
**Age**	Age of the employee	</br>
**Attrition**	Whether the employee left in the previous year or not	</br>
**BusinessTravel**	How frequently the employees travelled for business purposes in the last year	</br>
**Department**	Department in company	</br>
**DistanceFromHome**	Distance from home in kms	</br>
**Education**	Education Level</br>
        1 'Below College'
		2 'College'
		3 'Bachelor'
		4 'Master'
		5 'Doctor'</br>
**EducationField**	Field of education	</br>
**EmployeeCount**	Employee count	</br>
**EmployeeNumber**	Employee number/id	</br>
**EnvironmentSatisfaction**	Work Environment Satisfaction Level</br>
        1 'Low'
		2 'Medium'
		3 'High'
		4 'Very High'</br>
**Gender**	Gender of employee	</br>
**JobInvolvement**	Job Involvement Level</br>
        1 'Low'
		2 'Medium'
		3 'High'
		4 'Very High'</br>
**JobLevel**	Job level at company on a scale of 1 to 5	</br>
**JobRole**	Name of job role in company	</br>
**JobSatisfaction**	Job Satisfaction Level</br>
        1 'Low'
		2 'Medium'
		3 'High'
		4 'Very High'</br>
**MaritalStatus**	Marital status of the employee	</br>
**MonthlyIncome**	Monthly income in rupees per month	</br>
**NumCompaniesWorked**	Total number of companies the employee has worked for	</br>
**Over18**	Whether the employee is above 18 years of age or not	</br>
**PercentSalaryHike**	Percent salary hike for last year	</br>
**PerformanceRating**	Performance rating for last year	</br>
        1 'Low'
		2 'Good'
		3 'Excellent'
		4 'Outstanding'</br>
**RelationshipSatisfaction**	Relationship satisfaction level</br>
        1 'Low'
		2 'Medium'
		3 'High'
		4 'Very High'</br>
**StandardHours**	Standard hours of work for the employee	</br>
**StockOptionLevel**	Stock option level of the employee	</br>
**TotalWorkingYears**	Total number of years the employee has worked so far	</br>
**TrainingTimesLastYear**	Number of times training was conducted for this employee last year	</br>
**WorkLifeBalance**	Work life balance level	</br>
        1 'Bad'
		2 'Good'
		3 'Better'
		4 'Best'</br>
**YearsAtCompany**	Total number of years spent at the company by the employee	
**YearsSinceLastPromotion**	Number of years since last promotion	
**YearsWithCurrManager**	Number of years under current manager	

In [1]:
# Common imports
import sqlite3
import pandas as pd
import numpy as np
import os
import seaborn as sns
sns.set()
# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [3]:
conn = sqlite3.connect("data/hr_data.db")
employee_survey = pd.read_sql_query("select * from employee_survey_data", conn)
general_data = pd.read_sql_query("select * from general_data", conn)
manager_survey = pd.read_sql_query("select * from manager_survey_data", conn)
in_time = pd.read_sql_query("select * from in_time", conn)
out_time = pd.read_sql_query("select * from out_time", conn)

all_data = pd.read_sql_query("select *, Case when Attrition ='Yes' then 1 else 0 end as 'Attr' from general_data g join employee_survey_data e on g.EmployeeID=e.EmployeeID join manager_survey_data m on g.EmployeeID=m.EmployeeID Where EnvironmentSatisfaction Not Like 'NA' AND JobSatisfaction Not Like 'NA' AND WorkLifeBalance Not Like 'NA' AND TotalWorkingYears Not Like 'NA' AND NumCompaniesWorked Not Like 'NA'", conn)
removed_data = pd.read_sql_query("select * from general_data g join employee_survey_data e on g.EmployeeID=e.EmployeeID join manager_survey_data m on g.EmployeeID=m.EmployeeID Where EnvironmentSatisfaction Like 'NA' or JobSatisfaction Like 'NA' or WorkLifeBalance Like 'NA' or TotalWorkingYears Like 'NA' or NumCompaniesWorked Like 'NA' ", conn)


In [4]:
all_data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,YearsSinceLastPromotion,YearsWithCurrManager,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,EmployeeID,JobInvolvement,PerformanceRating,Attr
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,0,0,1,3,4,2,1,3,3,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,1,4,2,3,2,4,2,2,4,1
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,0,3,3,2,2,1,3,3,3,0
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,7,5,4,4,4,3,4,2,3,0
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,0,4,5,4,1,3,5,3,3,0


In [5]:
## The list of removed rows because they contained an NA
#removed_data.head()

In [6]:
## Converting numbers stored as objects over to numbers

all_data['JobSatisfaction'] = all_data['JobSatisfaction'].astype(np.int64)
all_data['WorkLifeBalance'] = all_data['WorkLifeBalance'].astype(np.int64)
all_data['EnvironmentSatisfaction'] = all_data['EnvironmentSatisfaction'].astype(np.int64)

In [7]:
#drop the useless columns:
all_data.drop(['EmployeeCount','EmployeeID','StandardHours','Over18'],axis=1, inplace = True)

In [8]:
all_data.drop_duplicates(keep = 'first',inplace=True) 

In [9]:
hr_data_uc = all_data[['Age','DistanceFromHome','Education', 
'JobLevel',
'MonthlyIncome',
'NumCompaniesWorked',
'PercentSalaryHike',
'StockOptionLevel',
'TotalWorkingYears',
'TrainingTimesLastYear',
'YearsAtCompany',
'YearsSinceLastPromotion',
'YearsWithCurrManager',
'EnvironmentSatisfaction',
'JobSatisfaction',
'WorkLifeBalance',
'JobInvolvement',
'PerformanceRating'   ]].copy()
hr_data_uc.head()

,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,51,6,2,1,131160,1,11,0,1,6,1,0,0,3,4,2,3,3
1,31,10,1,1,41890,0,23,1,6,3,5,1,4,3,2,4,2,4
2,32,17,4,4,193280,1,15,3,5,2,5,0,3,2,2,1,3,3
3,38,2,5,3,83210,3,11,3,13,5,8,7,5,4,4,3,2,3
4,32,10,1,1,23420,4,12,2,9,2,6,0,4,4,1,3,3,3


In [10]:
hr_data_cat = all_data[['Attrition', 'BusinessTravel','Department',
                       'EducationField','Gender','JobRole',
                       'MaritalStatus']].copy()
hr_data_cat.head()


,Attrition,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus
0,No,Travel_Rarely,Sales,Life Sciences,Female,Healthcare Representative,Married
1,Yes,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Single
2,No,Travel_Frequently,Research & Development,Other,Male,Sales Executive,Married
3,No,Non-Travel,Research & Development,Life Sciences,Male,Human Resources,Married
4,No,Travel_Rarely,Research & Development,Medical,Male,Sales Executive,Single


In [11]:
Num_val = {'Yes':1, 'No':0}
hr_data_cat['Attrition'] = hr_data_cat["Attrition"].apply(lambda x: Num_val[x])
hr_data_cat.head()

,Attrition,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus
0,0,Travel_Rarely,Sales,Life Sciences,Female,Healthcare Representative,Married
1,1,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Single
2,0,Travel_Frequently,Research & Development,Other,Male,Sales Executive,Married
3,0,Non-Travel,Research & Development,Life Sciences,Male,Human Resources,Married
4,0,Travel_Rarely,Research & Development,Medical,Male,Sales Executive,Single


In [12]:
hr_data_cat = pd.get_dummies(hr_data_cat)
hr_data_cat.head()

,Attrition,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
0,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,1,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
2,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [13]:
hr_data_final = pd.concat([hr_data_uc, hr_data_cat], axis=1)
hr_data_final.head()

,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
0,51,6,2,1,131160,1,11,0,1,6,...,0,0,0,0,0,0,0,0,1,0
1,31,10,1,1,41890,0,23,1,6,3,...,0,0,0,0,1,0,0,0,0,1
2,32,17,4,4,193280,1,15,3,5,2,...,0,0,0,0,0,1,0,0,1,0
3,38,2,5,3,83210,3,11,3,13,5,...,0,0,0,0,0,0,0,0,1,0
4,32,10,1,1,23420,4,12,2,9,2,...,0,0,0,0,0,1,0,0,0,1


In [14]:
#corr_cols = hr_data_final
#corr_cols =all_data

In [15]:
#corr = corr_cols.corr()
#plt.figure(figsize=(30,12))
#sns.heatmap(corr,annot=True)
#plt.show()

### Neural

In [16]:
X = hr_data_final.drop(["Attrition"], axis=1)
y = hr_data_final["Attrition"]  #column to make the decision on.

In [17]:
print(X.shape)
print(y.shape)

(1470, 44)
(1470,)


In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [19]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1102, 44)
(1102,)
(368, 44)
(368,)


In [20]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)

C:\Users\Isaac\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [21]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\Isaac\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.
C:\Users\Isaac\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  


In [22]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [23]:
from tensorflow.keras.layers import Dense
number_inputs = 44
number_hidden_nodes = 7 #Test to see if number of hidden layers 
# model.add(Dense(units=number_hidden_nodes,
#                 activation='relu', input_dim=number_inputs))
# model.add(Dense(units=number_hidden_nodes,
#                 activation='relu', input_dim=number_inputs))
# model.add(Dense(units=number_hidden_nodes,
#                 activation='relu', input_dim=number_inputs))
model.add(Dense(units=number_hidden_nodes, input_dim=number_inputs))
model.add(Dense(units=number_hidden_nodes, input_dim=number_inputs))
model.add(Dense(units=number_hidden_nodes, input_dim=number_inputs))
model.add(Dense(units=number_hidden_nodes, input_dim=number_inputs))
#model.add(Dense(units=number_hidden_nodes, input_dim=number_inputs))

Instructions for updating:
Colocations handled automatically by placer.


In [24]:
number_classes = 2
#model.add(Dense(units=5, activation='relu')) #deep learning addition
model.add(Dense(units=number_classes, activation='softmax'))

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 7)                 315       
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 16        
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])




In [27]:
# Fit (train) the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 0s - loss: 0.8023 - acc: 0.5445
Epoch 2/100
 - 0s - loss: 0.5509 - acc: 0.7450
Epoch 3/100
 - 0s - loss: 0.4666 - acc: 0.8249
Epoch 4/100
 - 0s - loss: 0.4258 - acc: 0.8439
Epoch 5/100
 - 0s - loss: 0.4022 - acc: 0.8439
Epoch 6/100
 - 0s - loss: 0.3898 - acc: 0.8439
Epoch 7/100
 - 0s - loss: 0.3834 - acc: 0.8439
Epoch 8/100
 - 0s - loss: 0.3780 - acc: 0.8466
Epoch 9/100
 - 0s - loss: 0.3728 - acc: 0.8439
Epoch 10/100
 - 0s - loss: 0.3695 - acc: 0.8475
Epoch 11/100
 - 0s - loss: 0.3658 - acc: 0.8475
Epoch 12/100
 - 0s - loss: 0.3638 - acc: 0.8475
Epoch 13/100
 - 0s - loss: 0.3637 - acc: 0.8512
Epoch 14/100
 - 0s - loss: 0.3605 - acc: 0.8494
Epoch 15/100
 - 0s - loss: 0.3606 - acc: 0.8485
Epoch 16/100
 - 0s - loss: 0.3602 - acc: 0.8530
Epoch 17/100
 - 0s - loss: 0.3602 - acc: 0.8548
Epoch 18/100
 - 0s - loss: 0.3590 - acc: 0.8512
Epoch 19/100
 - 0s - loss: 0.3589 - acc: 0.8494
Epoch 20/100
 - 0s - loss: 0.3592 - acc: 0.8521
Epoch 21/100
 - 0s - loss: 0.3581 - acc: 0.8503
E

In [28]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 0s - loss: 0.4037 - acc: 0.8342
Loss: 0.40367565854736, Accuracy: 0.83423912525177
